In [2]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)

In [6]:
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 0.0667,  0.0357,  0.1419,  0.1640, -0.1120],
          [ 0.1489,  0.0517, -0.0071,  0.1824,  0.1067],
          [ 0.0764,  0.0485,  0.1004, -0.0234,  0.0819],
          [-0.0163, -0.0198, -0.0569, -0.1769, -0.1567],
          [-0.1167,  0.0235, -0.0667, -0.1726, -0.0213]]],


        [[[-0.0787,  0.1017, -0.1796, -0.1655, -0.0449],
          [ 0.0863,  0.1568,  0.0179, -0.1810,  0.0276],
          [-0.1481,  0.1420,  0.1565, -0.0609,  0.1882],
          [ 0.1435, -0.1081,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.1547,  0.1597,  0.1284, -0.1527]]],


        [[[-0.1017, -0.1946, -0.1383,  0.0648, -0.1407],
          [-0.1090, -0.0511,  0.0697,  0.0737, -0.0771],
          [-0.0069,  0.0570,  0.0786, -0.0604,  0.1978],
          [ 0.0857,  0.1504,  0.0641, -0.1069, -0.1769],
          [ 0.0969, -0.0036,  0.1420, -0.0533, -0.0329]]],


        [[[ 0.1524, -0.0125, -0.0613,  0.0719,  0.0128],
          [ 0.1186,  0.0125,  0.0843,  0.1

In [7]:
print(list(module.named_buffers()))

[]


In [8]:
prune.random_unstructured(module, name="weight", amount=0.3)


Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

In [9]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 0.0722,  0.0131,  0.0242, -0.1499, -0.1643,  0.0177], device='cuda:0',
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0667,  0.0357,  0.1419,  0.1640, -0.1120],
          [ 0.1489,  0.0517, -0.0071,  0.1824,  0.1067],
          [ 0.0764,  0.0485,  0.1004, -0.0234,  0.0819],
          [-0.0163, -0.0198, -0.0569, -0.1769, -0.1567],
          [-0.1167,  0.0235, -0.0667, -0.1726, -0.0213]]],


        [[[-0.0787,  0.1017, -0.1796, -0.1655, -0.0449],
          [ 0.0863,  0.1568,  0.0179, -0.1810,  0.0276],
          [-0.1481,  0.1420,  0.1565, -0.0609,  0.1882],
          [ 0.1435, -0.1081,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.1547,  0.1597,  0.1284, -0.1527]]],


        [[[-0.1017, -0.1946, -0.1383,  0.0648, -0.1407],
          [-0.1090, -0.0511,  0.0697,  0.0737, -0.0771],
          [-0.0069,  0.0570,  0.0786, -0.0604,  0.1978],
          [ 0.0857,  0.1504,  0.0641, -0.1069, -0.1769],
          [ 0.0

In [10]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 0., 1., 0., 0.],
          [1., 0., 1., 0., 0.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 0., 0., 1., 1.]]],


        [[[0., 0., 1., 1., 0.],
          [0., 1., 0., 0., 1.],
          [1., 1., 0., 0., 1.],
          [1., 0., 1., 1., 1.],
          [1., 0., 0., 1., 1.]]],


        [[[0., 1., 1., 1., 1.],
          [0., 0., 1., 1., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 0., 1., 1.],
          [0., 1., 1., 1., 1.]]],


        [[[0., 1., 1., 1., 0.],
          [1., 1., 1., 0., 0.],
          [1., 0., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 0., 0.]]],


        [[[0., 1., 0., 1., 1.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 0., 0.],
          [0., 1., 1., 0., 1.]]],


        [[[0., 1., 1., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 0., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 1., 1., 1.]]]], 

In [11]:
print(module.weight)

tensor([[[[ 0.0667,  0.0000,  0.1419,  0.0000, -0.0000],
          [ 0.1489,  0.0000, -0.0071,  0.0000,  0.0000],
          [ 0.0000,  0.0485,  0.1004, -0.0234,  0.0819],
          [-0.0163, -0.0198, -0.0569, -0.1769, -0.1567],
          [-0.1167,  0.0000, -0.0000, -0.1726, -0.0213]]],


        [[[-0.0000,  0.0000, -0.1796, -0.1655, -0.0000],
          [ 0.0000,  0.1568,  0.0000, -0.0000,  0.0276],
          [-0.1481,  0.1420,  0.0000, -0.0000,  0.1882],
          [ 0.1435, -0.0000,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.0000,  0.0000,  0.1284, -0.1527]]],


        [[[-0.0000, -0.1946, -0.1383,  0.0648, -0.1407],
          [-0.0000, -0.0000,  0.0697,  0.0737, -0.0000],
          [-0.0069,  0.0570,  0.0786, -0.0604,  0.0000],
          [ 0.0857,  0.1504,  0.0000, -0.1069, -0.1769],
          [ 0.0000, -0.0036,  0.1420, -0.0533, -0.0329]]],


        [[[ 0.0000, -0.0125, -0.0613,  0.0719,  0.0000],
          [ 0.1186,  0.0125,  0.0843,  0.0000,  0.0000],
          [-0.0193,

In [12]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x7f042c15af20>)])


In [13]:
prune.l1_unstructured(module, name="bias", amount=3)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

In [14]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.0667,  0.0357,  0.1419,  0.1640, -0.1120],
          [ 0.1489,  0.0517, -0.0071,  0.1824,  0.1067],
          [ 0.0764,  0.0485,  0.1004, -0.0234,  0.0819],
          [-0.0163, -0.0198, -0.0569, -0.1769, -0.1567],
          [-0.1167,  0.0235, -0.0667, -0.1726, -0.0213]]],


        [[[-0.0787,  0.1017, -0.1796, -0.1655, -0.0449],
          [ 0.0863,  0.1568,  0.0179, -0.1810,  0.0276],
          [-0.1481,  0.1420,  0.1565, -0.0609,  0.1882],
          [ 0.1435, -0.1081,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.1547,  0.1597,  0.1284, -0.1527]]],


        [[[-0.1017, -0.1946, -0.1383,  0.0648, -0.1407],
          [-0.1090, -0.0511,  0.0697,  0.0737, -0.0771],
          [-0.0069,  0.0570,  0.0786, -0.0604,  0.1978],
          [ 0.0857,  0.1504,  0.0641, -0.1069, -0.1769],
          [ 0.0969, -0.0036,  0.1420, -0.0533, -0.0329]]],


        [[[ 0.1524, -0.0125, -0.0613,  0.0719,  0.0128],
          [ 0.1186,  0.0125,  0.0843,

In [15]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[1., 0., 1., 0., 0.],
          [1., 0., 1., 0., 0.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 0., 0., 1., 1.]]],


        [[[0., 0., 1., 1., 0.],
          [0., 1., 0., 0., 1.],
          [1., 1., 0., 0., 1.],
          [1., 0., 1., 1., 1.],
          [1., 0., 0., 1., 1.]]],


        [[[0., 1., 1., 1., 1.],
          [0., 0., 1., 1., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 0., 1., 1.],
          [0., 1., 1., 1., 1.]]],


        [[[0., 1., 1., 1., 0.],
          [1., 1., 1., 0., 0.],
          [1., 0., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 0., 0.]]],


        [[[0., 1., 0., 1., 1.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 0., 0.],
          [0., 1., 1., 0., 1.]]],


        [[[0., 1., 1., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 0., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 1., 1., 1.]]]], 

In [16]:
print(module.bias)

tensor([ 0.0722,  0.0000,  0.0000, -0.1499, -0.1643,  0.0000], device='cuda:0',
       grad_fn=<MulBackward0>)


In [17]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x7f042c15af20>), (1, <torch.nn.utils.prune.L1Unstructured object at 0x7f042c159720>)])


In [18]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# As we can verify, this will zero out all the connections corresponding to
# 50% (3 out of 6) of the channels, while preserving the action of the
# previous mask.
print(module.weight)

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[-0.0000,  0.0000, -0.1796, -0.1655, -0.0000],
          [ 0.0000,  0.1568,  0.0000, -0.0000,  0.0276],
          [-0.1481,  0.1420,  0.0000, -0.0000,  0.1882],
          [ 0.1435, -0.0000,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.0000,  0.0000,  0.1284, -0.1527]]],


        [[[-0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000,

In [19]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # select out the correct hook
        break

print(list(hook))  # pruning history in the container

[<torch.nn.utils.prune.RandomUnstructured object at 0x7f042c15af20>, <torch.nn.utils.prune.LnStructured object at 0x7f042c1596c0>]


In [20]:
print(model.state_dict().keys())

odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


In [21]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.0667,  0.0357,  0.1419,  0.1640, -0.1120],
          [ 0.1489,  0.0517, -0.0071,  0.1824,  0.1067],
          [ 0.0764,  0.0485,  0.1004, -0.0234,  0.0819],
          [-0.0163, -0.0198, -0.0569, -0.1769, -0.1567],
          [-0.1167,  0.0235, -0.0667, -0.1726, -0.0213]]],


        [[[-0.0787,  0.1017, -0.1796, -0.1655, -0.0449],
          [ 0.0863,  0.1568,  0.0179, -0.1810,  0.0276],
          [-0.1481,  0.1420,  0.1565, -0.0609,  0.1882],
          [ 0.1435, -0.1081,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.1547,  0.1597,  0.1284, -0.1527]]],


        [[[-0.1017, -0.1946, -0.1383,  0.0648, -0.1407],
          [-0.1090, -0.0511,  0.0697,  0.0737, -0.0771],
          [-0.0069,  0.0570,  0.0786, -0.0604,  0.1978],
          [ 0.0857,  0.1504,  0.0641, -0.1069, -0.1769],
          [ 0.0969, -0.0036,  0.1420, -0.0533, -0.0329]]],


        [[[ 0.1524, -0.0125, -0.0613,  0.0719,  0.0128],
          [ 0.1186,  0.0125,  0.0843,

In [22]:
    print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 1., 1., 0.],
          [0., 1., 0., 0., 1.],
          [1., 1., 0., 0., 1.],
          [1., 0., 1., 1., 1.],
          [1., 0., 0., 1., 1.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[0., 1., 0., 1., 1.],
          [0., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 0., 0.],
          [0., 1., 1., 0., 1.]]],


        [[[0., 1., 1., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 0., 1., 1.],
          [1., 1., 1., 0., 1.],
          [1., 1., 1., 1., 1.]]]], 

In [23]:
print(module.weight)

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[-0.0000,  0.0000, -0.1796, -0.1655, -0.0000],
          [ 0.0000,  0.1568,  0.0000, -0.0000,  0.0276],
          [-0.1481,  0.1420,  0.0000, -0.0000,  0.1882],
          [ 0.1435, -0.0000,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.0000,  0.0000,  0.1284, -0.1527]]],


        [[[-0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000, -0.0000,  0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000,

In [24]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('bias_orig', Parameter containing:
tensor([ 0.0722,  0.0131,  0.0242, -0.1499, -0.1643,  0.0177], device='cuda:0',
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[-0.0000,  0.0000, -0.1796, -0.1655, -0.0000],
          [ 0.0000,  0.1568,  0.0000, -0.0000,  0.0276],
          [-0.1481,  0.1420,  0.0000, -0.0000,  0.1882],
          [ 0.1435, -0.0000,  0.1197,  0.0508,  0.1661],
          [-0.0524, -0.0000,  0.0000,  0.1284, -0.1527]]],


        [[[-0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0

In [25]:
print(list(module.named_buffers()))

[('bias_mask', tensor([1., 0., 0., 1., 1., 0.], device='cuda:0'))]


In [26]:
new_model = LeNet()
for name, module in new_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(new_model.named_buffers()).keys())  # to verify that all masks exist

dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask', 'fc3.weight_mask'])


In [27]:
model = LeNet()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

In [28]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

Sparsity in conv1.weight: 7.33%
Sparsity in conv2.weight: 12.79%
Sparsity in fc1.weight: 22.20%
Sparsity in fc2.weight: 12.34%
Sparsity in fc3.weight: 9.05%
Global sparsity: 20.00%


In [29]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0
        return mask

In [30]:
def foobar_unstructured(module, name):
    """Prunes tensor corresponding to parameter called `name` in `module`
    by removing every other entry in the tensors.
    Modifies module in place (and also return the modified module)
    by:
    1) adding a named buffer called `name+'_mask'` corresponding to the
    binary mask applied to the parameter `name` by the pruning method.
    The parameter `name` is replaced by its pruned version, while the
    original (unpruned) parameter is stored in a new parameter named
    `name+'_orig'`.

    Args:
        module (nn.Module): module containing the tensor to prune
        name (string): parameter name within `module` on which pruning
                will act.

    Returns:
        module (nn.Module): modified (i.e. pruned) version of the input
            module

    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module

In [31]:
model = LeNet()
foobar_unstructured(model.fc3, name='bias')

print(model.fc3.bias_mask)

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
